# 0.Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
import seaborn as sns

In [3]:
import pandas as pd
import joblib
import warnings;warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

In [4]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

# 1.Data Load

In [5]:
cus = pd.read_csv("../data/cus_info.csv")
iem = pd.read_csv("../data/iem_info.csv")
hist = pd.read_csv("../data/stk_bnc_hist.csv")
train = pd.read_csv("../data/stk_hld_train.csv")
test = pd.read_csv("../data/stk_hld_test.csv")
submission = pd.read_csv("../data/sample_submission.csv")
df_date = pd.read_csv('../data/날짜별_개장여부.csv',encoding='cp949')
sell_dt = pd.read_csv('../data/tk_hld_newtrain.csv')

In [6]:
#cus.columns = ['계좌ID','성별','연령대','투자성향','자산구간','주거래상품군','Life Style','서비스등급','총 투자기간','주거래업종구분']
display(cus.head())
print(cus.shape)

,act_id,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd
0,64aae8dd71e5c0761000db9f9a6779e504e7a4aa9dc097...,1,4,99,1,2,3,3,6,16
1,5f7c3a8f37d9c268d06130ff0be5d32a1b9ef68c13049f...,1,6,4,4,2,5,2,6,1
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2,7,4,3,2,5,5,6,9
3,6d497facfa1ea5901b827335553331f8555fec02a8184f...,2,6,4,4,8,5,3,6,16
4,b727c78d2cfc246c97b677f29a034399a0c7e7873fff44...,1,5,2,2,2,5,5,5,16


(10000, 10)


In [7]:
#iem.columns = ['종목코드','종목한글명','종목업종','시가총액,규모유형','시장구분']
display(iem.head())
print(iem.shape)

,iem_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,A000020,동화약품,14,2,99
1,A000080,하이트진로,14,2,1
2,A000180,성창기업지주,14,3,99
3,A000227,유유제약2우B,14,99,99
4,A000325,노루홀딩스우,14,99,99


(3078, 5)


In [8]:
#hist.columns = ['계좌ID','기준일자','종목코드','잔고수량','잔고금액','액면가']
display(hist.head())
print(hist.shape)

,act_id,bse_dt,iem_cd,bnc_qty,tot_aet_amt,stk_par_pr
0,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200820,A008770,40.0,2828000.0,5000.0
1,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A008770,20.0,1390000.0,5000.0
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20160104,A005940,311.0,2982490.0,5000.0
3,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200814,A005930,40.0,2320000.0,100.0
4,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A005930,20.0,1028000.0,100.0


(2573839, 6)


In [9]:
#train.columns = ['계좌ID','종목코드','매수일자','보유기간(일)']
display(train.head())
print(train.shape)

,act_id,iem_cd,byn_dt,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324


(681472, 4)


In [10]:
#test.columns = ['계좌ID','종목코드','매수일자','과거보유일','제출ID','보유기간(일)']
display(test.head())
print(test.shape)

,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0


(70596, 6)


In [11]:
display(submission.head())
print(submission.shape)

,submit_id,hold_d
0,IDX00001,0
1,IDX00002,0
2,IDX00003,0
3,IDX00004,0
4,IDX00005,0


(70596, 2)


In [12]:
display(sell_dt.head())
print(sell_dt.shape)

,Unnamed: 0,act_id,iem_cd,byn_dt,hold_d,sell_dt
0,0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,2018-08-10
1,1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,2018-01-31,80,2018-05-31
2,2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,2018-05-17,5,2018-05-25
3,3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,2020-11-12,22,2020-12-14
4,4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,2018-09-05,324,2020-01-03


(681472, 6)
